# [CSCI 3397/PSYC 3317] Lab 6b: Optimization and PyTorch Basics

**Posted:** Thursday, February 24, 2022

**Due:** Thursday, March 3, 2022

__Total Points__: 2 pts

__Submission__: please rename the .ipynb file as __\<your_username\>\_lab6b.ipynb__ before you submit it to canvas. Example: weidf_lab6b.ipynb.

# 1. Optimization Basics


Let's implement the gradient descend method to find the minimum value for the function $g(x) = 0.066x^4-0.32x^3-0.85x^2+ 4.2x+8.2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## function definition
def g(x):
  # return the value of the defined g(x) at value x
  return 8.2 + 4.2*x**1 -0.85*x**2 -0.32*x**3+ 0.066*x**4

def Dg(x):
  # return the value of the gradient of the defined g(x) at value x
  return 4.2 -1.7*x -0.96*x**2+ 0.264*x**3

## gradient descent
def optimizer_grad_descent(x0, grad, alpha = 0.1, num_step = 100, x_ran=None):
  for x in range(num_step):
    # gradient descent update    
    x0 = x0 - alpha * grad(x0)
    if x_ran is not None:
      # clip the value
      x0 = np.clip(x0, x_ran[0], x_ran[1])
  return x0

# find the minimum value within x_ran, starting from x0
x_ran = [-4, 6]
x0 = 5.5
x1 = optimizer_grad_descent(x0, Dg, alpha = 0.1, x_ran = x_ran)

# plot result
print('initial result: x=%.2f, g(x)=%.2f' %(x0, g(x0)))
print('final result: x=%.2f, g(x)=%.2f' %(x1, g(x1)))
xx = np.linspace(*x_ran, 100)
plt.plot(xx, g(xx))
plt.plot(x0,g(x0), 'ro')
plt.plot(x1,g(x1), 'kx', markersize=20)
plt.show()

# (2 pts) Exercise
Let's implement a better optimization method: "gradient descend with momentum" method described in [distill.pub paper](https://distill.pub/2017/momentum/). 
The high-level idea is that instead of trusting the current gradient 100%, we can linearly combine the previous gradients (i.e., carry on the momemtum) and the current gradient. With this, it can stabilize the search moves (e.g., not affected by noisy local gradient) and get out of stuck of the local optima where the current graident is 0.

Here's the new Update formula: 
- $z^{k+1}=\beta z^{k}+ \dfrac{dg}{dx}(x^{k})$
- $x^{k+1}=x^{k+1} -\alpha z^{k+1}$

Note that, if $\beta=0$, the update rule is the same as the gradient descent method.

Let's implement it and find the global mimina for this function.

In [ ]:
## gradient descent with momemtum
def optimizer_grad_descent_momentum(x0, grad, alpha = 0.1, beta = 0.99, num_step = 100, x_ran=None):
    v0 = 0
    for x in range(num_step):
        # gradient descend with momentum update    
        ### Your code starts here
        
        ### Your code ends here        
        if x_ran is not None:
            # clip the value
            x0 = np.clip(x0, x_ran[0], x_ran[1])
    return x0

# find the minimum value within x_ran, starting from x0
x_ran = [-4, 6]
x0 = 5.5
x2 = optimizer_grad_descent_momentum(x0, Dg, x_ran = x_ran)

# plot result
print('initial result: x=%.2f, g(x)=%.2f' %(x0, g(x0)))
print('final result: x=%.2f, g(x)=%.2f' %(x1, g(x1)))
xx = np.linspace(*x_ran, 100)
plt.plot(xx, g(xx))
plt.plot(x0,g(x0), 'ro')
plt.plot(x2,g(x2), 'kx', markersize=20)
plt.show()

# 2. PyTorch Basics

We will explain how to implement machine learning building blocks (model, loss, and optimization) in PyTorch.

### What is PyTorch?
1. A Python GPU-accelerated tensor library (NumPy, but faster)
2. Differentiable Programming with dynamic computation graphs
3. Flexible and efficient **neural network** library
4. Python-first framework (easy to integrate with other Python libraries, debug, and extend)
  + Quick conversion from & to NumPy array, integration with other Python libs.
  + Your favorite Python debugger.
  + Adding custom ops with Python/c++ extension. 
  + Running in purely c++ environment with the c++ API.

Useful links:

+ PyTorch documentation: https://pytorch.org/docs/stable/index.html
  -  Most math operations can be found as `torch.*` or `Tensor.*`.
+ [Optional] PyTorch official tutorials: https://pytorch.org/tutorials/
  - Transfer learning tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
+ [Optional] PyTorch examples: https://github.com/pytorch/examples/
  - DCGAN, ImageNet training, Reinforcement Learning, etc.

### PyTorch Installation

In [3]:
# install torch and torchvision (a utility library for computer vision that provides many public datasets and pre-trained models)
# already installed on jupyterhub
# !pip install torch torchvision

## 2.1 GPU-accelerated Tensor Library

The syntax of the torch tensor library is similar to that of numpy.

In [ ]:
import torch

In [ ]:
# Create a 3x5 matrix filled with zeros
x = torch.zeros(3, 5)
print(x)

In [ ]:
# Create a 3x5 matrix filled with random values from a standard normal distribution
y = torch.randn(3, 5)
print(y)

In [ ]:
# Shape manipulations
print('\n.t()  (transpose): ')
print(y.t())

print('.reshape(5, 3): ')
print(y.reshape(5, 3))

In [ ]:
# Slicing
print(y[1:])

# Slicing + select every two elements
print(y[1:, ::2])

In [ ]:
# Basic arithmetics
print(x + 2)

In [ ]:
print(y * (x + 2))

In [ ]:
print((y * (x + 2)).exp())

#### GPU Acceleration (BC jupyterhub doesn't have GPU support)

Everything can be run on a GPU

First, let us create a [`torch.device`](https://pytorch.org/docs/stable/tensor_attributes.html#torch-device) object representing a GPU device.

### NumPy Bridge

Converting a `torch.Tensor` to a `np.ndarray` and vice versa is a breeze.

The `torch.Tensor` and `np.ndarray` will share their underlying memory locations (if the `torch.Tensor` is on CPU and `dtype` is the same), and changing one will change the other.

In [ ]:
import numpy as np

In [ ]:
# convert a torch tensor into a numpy array
x = torch.randn(5)
x_np1 = x.numpy()
x_np2 = np.asarray(x)

print(x)
print(x_np1)
print(x_np2)

In [ ]:
# convert a numpy array into a torch tensor

a = np.random.randn(3, 4)
a_pt = torch.as_tensor(a)
print(a)
print(a_pt)

In [ ]:
# the resulting CPU Tensor shares memory with the array!
# change the tensor array -> change the orignial numpy array
# if you want a different copy: a_pt = torch.as_tensor(a.copy())

a_pt[0] = -1
print(a)

## 2.2 Flexible and Efficient Neural Network Library

The [`torch.nn`](https://pytorch.org/docs/stable/nn.html) and [`torch.optim`](https://pytorch.org/docs/stable/optim.html) packages provide many efficient implementations of neural network components:
  + Affine layers and [activation functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
  + Normalization methods
  + [Initialization schemes](https://pytorch.org/docs/stable/nn.html#torch-nn-init)
  + [Loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
  + [Embeddings](https://pytorch.org/docs/stable/nn.html#sparse-layers)
  + [Distributed and Multi-GPU training](https://pytorch.org/docs/stable/nn.html#dataparallel-layers-multi-gpu-distributed)
  + [Gradient-based optimizers](https://pytorch.org/docs/stable/optim.html)
  + [Learning rate schedulers](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)
  + etc.

### (a1) Model: Linear Layer

We will use the [fully connected linear layer (`nn.Linear`)](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear). 
A fc layer performs an affine transform with a 2D weight parameter $\mathbf{w}$ and a 1D bias parameter $\mathbf{b}$:

$$ f(\mathbf{x}) = \mathbf{w}^\mathrm{T} \mathbf{x} + \mathbf{b}$$

In [ ]:
# all popular neural network layers
import torch.nn as nn 
# handy for simple functions
import torch.nn.functional as F

In [ ]:
# input x: 1D array of size 4
# output: 1D array of size 8
fc = nn.Linear(in_features=4, out_features=8)
print(fc)

In [ ]:
# It has two parameters, the weight and the bias
for name, p in fc.named_parameters():
    print('param name: {}\t shape: {}'.format(name, p.shape))

In [ ]:
fc.weight

In [ ]:
# These parameters by default have `requires_grad=True`, so they will collect gradients!
print(fc.bias)

### (a2) Model: Sigmoid Layer

$$ \sigma(x) = \frac{1}{1+e^{-x}}.$$

In [ ]:
# Let's construct an input tensor with 2 dimensions:
#   - batch dimensionsize: 2
#   - input size: 4
x = torch.randn(2, 4)

result_linear = fc(x)
result_logistic = F.sigmoid(result_linear)
print("linear layer output: range=[-\infty, \infty]")
print(result_linear)
print("logistic output: range=[0, 1]")
print(result_logistic)


### (b) Loss: Cross entropy and MSE Loss Function

[Documentation](https://pytorch.org/docs/stable/nn.functional.html)

In [ ]:
# generate arbitrary classification label
target = result_linear>0.1
target = target.to(torch.float) # need to convert it to float

# Let's try MSE loss
loss_mse = F.mse_loss(result_logistic, target)
loss_bce = F.binary_cross_entropy(result_logistic, target)
print(loss_mse, loss_bce)

### (c) Autograd and Optimizer

#### Gradient Computation
PyTorch keeps track of your computations and the gradient is automatically computed!

In [ ]:
# Compute gradients
loss_mse.backward()
print(fc.bias.grad)

#### Optimizer

We can code up a naive optimizer with manual gradient updates like before.

In [ ]:
# We can manually perform GD via a loop
print('bias before GD', fc.bias)
lr = 0.5
with torch.no_grad():  
    # this context manager tells PyTorch that we don't want ops inside to be 
    # tracked by autograd!
    # o/w PyTorch will try to automatically compute the gradient of this gradient operation too.
    for p in fc.parameters():
        p -= lr * p.grad
        
print('bias after one-step GD', fc.bias)

More easily, we can use the provided [`torch.optim`](https://pytorch.org/docs/stable/optim.html#torch.optim) optimizers (e.g. GD+momentum and many advanced optimizers). We will see how to use the [`torch.optim.SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) optimizer in a second!

# 3. Logistic Regression in PyTorch

## (a) Data
- download and pre-procoss the dataset (x,y)
- divide them into train, val, and test sets in a 6:2:2 ratio
- convert from numpy to pytorch tensor

In [ ]:
# download data
import numpy as np
import pandas as pd
import torch

url = 'https://raw.githubusercontent.com/BlohmLab/MLtutorials/week3/data/marks.txt'
data = pd.read_csv(url, header=None)
Y = np.array(data.iloc[:,-1]).astype(np.float32).reshape([-1,1])

# by default, numpy arrays are float64, but pytorch tensor wants float32
X = np.array(data.iloc[:,:-1]).astype(np.float32)
# normalize the data for better learning
X = (X-X.mean(axis=0))/X.std(axis=0)

def data_split(N, ratio=[6,2,2]):
  # generate a shuffle array
  shuffle_idx = np.arange(N)
  np.random.shuffle(shuffle_idx)
  # divide into train-val-test by the ratio
  data_split = (np.cumsum(ratio)/float(sum(ratio))*N).astype(int)
  out_idx = [None] * len(ratio)
  out_idx[0] = shuffle_idx[:data_split[0]]
  for i in range(1,len(ratio)):
    out_idx[i] = shuffle_idx[data_split[i-1] : data_split[i]]
  return out_idx  

train_idx, val_idx, test_idx = data_split(len(Y))

X_train, Y_train = X[train_idx], Y[train_idx]
X_val, Y_val = X[val_idx], Y[val_idx]
X_test, Y_test = X[test_idx], Y[test_idx]

#### TODO: convert variables into pytorch tensors
X_train_pt, Y_train_pt = torch.as_tensor(X_train), torch.as_tensor(Y_train)
X_val_pt, Y_val_pt = torch.as_tensor(X_val), torch.as_tensor(Y_val)
X_test_pt, Y_test_pt = torch.as_tensor(X_test), torch.as_tensor(Y_test)
print(X_train_pt.size())

In [ ]:
def plot_logistic_regression(fig, param_W=None, param_b=None, x=X, y=Y, subp=111, title='train'):
    # plot original data
    X_admitted = x[y==1,:]
    X_rejected = x[y==0,:]

    ax = fig.add_subplot(subp)
    ax.scatter(X_admitted[:,0],X_admitted[:,1])
    ax.scatter(X_rejected[:,0],X_rejected[:,1])
    ax.set_xlabel('Mark 0')
    ax.set_ylabel('Mark 1')
    ax.legend(('Accept','Reject'))
    
    if param_W is not None:
        # plot the decision boundary
        xx = np.linspace(-2, 2,100)
        yy = -param_W[0]/param_W[1]*xx - param_b/param_W[1]
        ax.plot(xx,yy,'g-')
        plt.title(title)

fig = plt.figure()
plot_logistic_regression(fig, None, None, X_train, Y_train[:,0], 111, 'train data')
plt.show()        

## (b) Logistic Regression Model

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        # define the layers and parameters
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # this function will be called to process the input data
        outputs = F.sigmoid(self.linear(x))
        return outputs

# create a model (parameters are initialized)
# input (size = 2): two exam scores (the bias param will take care of constant term)
# ouput (size = 1): accept or not
model = LogisticRegression(input_dim = 2, output_dim = 1)

# upload the model to GPU
# model.cuda()

## (c) Optimizer
We will use all the data and SGD becomes the same as the gradient descent (GD).

In [ ]:
lr_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

## (d) Training
Well, only 4 important lines!

In [ ]:
num_iter = 150
# training loop
for ii in range(num_iter):
    # 1. forward pass
    Y_hat = model(X_train_pt)
    
    # 2. compute loss
    loss = F.mse_loss(Y_hat, Y_train_pt)
    
    # 3. compute gradients
    loss.backward()
    
    # 4. gradient update
    optimizer.step()
    
    # add some printing
    if ii % 10 == 0:
        print('iteration {}\tloss {:.5f}'.format(ii, loss))

# if the torch tensor has "require_grad", need to detach it first
ww = model.linear.weight.detach().numpy()[0]
bb = model.linear.bias.detach().numpy()[0]

## (e) Evaluation
We will first compute the accuray and plot the predicted decision boundary. Note that the model that fits the training data well, may still have much error on the val and test data.

In [ ]:
def compute_accuracy(model, x=X, y=Y):
    """function that compares predicted y to true y and returns accuracy"""
    y_pred = model(x)>0.5
    accuracy = (y_pred == y).sum()/len(y)
    return accuracy 

print('Train acc:', compute_accuracy(model, X_train_pt, Y_train_pt))
print('Val acc:', compute_accuracy(model, X_val_pt, Y_test_pt))
print('Test acc:', compute_accuracy(model, X_val_pt, Y_test_pt))

In [ ]:
fig = plt.figure()
plot_logistic_regression(fig, ww, bb, X_train, Y_train[:,0], 131, 'train')
plot_logistic_regression(fig, ww, bb, X_val, Y_val[:,0], 132, 'val')
plot_logistic_regression(fig, ww, bb, X_test, Y_test[:,0], 133, 'test')
plt.show()

